# Определение возраста покупателей

Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:

- Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
- Контролировать добросовестность кассиров при продаже алкоголя. 

Основные задачи:
- Построить модель, которая по фотографии определит приблизительный возраст человека. В нашем распоряжении набор фотографий людей с указанием возраста
- Добиться значения MAE на тестовой выборке не больше 8.

*Постройте и обучите свёрточную нейронную сеть на датасете с фотографиями людей. Добейтесь значения MAE на тестовой выборке не больше 8.
Функцию загрузки тестовой выборки load_test(path) напишите самостоятельно. Вместе со старыми функциями в коде должны быть:
load_train(path),
load_test(path),
create_model(input_shape),
train_model(model, train_data, test_data, batch_size, epochs, steps_per_epoch, validation_steps).*

## Исследовательский анализ данных

### Импорт библиотек

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random
import seaborn as sns

from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import accuracy_score
from PIL import Image
from IPython.display import display

### Загрузка данных

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
datagen = ImageDataGenerator(rescale=1./255)
datagen_flow = datagen.flow_from_dataframe(
                                           dataframe=labels,
                                           directory='/datasets/faces/final_files/',
                                           x_col='file_name',
                                           y_col='real_age',
                                           target_size=(224, 224),
                                           batch_size=32,
                                           class_mode='raw',
                                           seed=12345)

### Знакомство с данными

## Обучение модели

from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from sklearn.metrics import accuracy_score
import numpy as np
 
def load_train(path):
    datagen = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rescale=1/255.)

    train_datagen_flow = datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=16,
        class_mode='sparse',
        seed=12345)

    return train_datagen_flow
 
def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape,
                        #classes=1000,
                        include_top=False,
                        weights='imagenet'
                       )
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(12, activation='softmax'))
    optimizer = Adam(lr=0.00005)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model
 

def train_model(model, train_data, test_data, batch_size=None, epochs=3,
                steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data,
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)

    return model

2022-07-21 08:37:33.998052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2022-07-21 08:37:34.318023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Using TensorFlow backend.
Found 5694 validated image filenames.
Found 1897 validated image filenames.
2022-07-21 08:37:45.066868: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-07-21 08:37:45.779578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-21 08:37:45.779678: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:45.779710: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:37:46.176033: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-21 08:37:46.280479: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-21 08:37:47.040109: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-21 08:37:47.091263: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2022-07-21 08:37:47.091410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-07-21 08:37:47.096102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2022-07-21 08:37:47.096507: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-21 08:37:47.305573: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2022-07-21 08:37:47.308022: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x489e9f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-21 08:37:47.308055: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-21 08:37:47.665354: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4924a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-07-21 08:37:47.665394: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2022-07-21 08:37:47.667882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-21 08:37:47.667944: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:47.667954: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:37:47.667993: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-21 08:37:47.668004: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-21 08:37:47.668014: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-21 08:37:47.668024: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2022-07-21 08:37:47.685182: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:47.668032: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-07-21 08:37:47.672478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2022-07-21 08:37:52.829148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-21 08:37:52.829193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2022-07-21 08:37:52.829201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2022-07-21 08:37:52.843079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 30509 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 178 steps, validate for 60 steps
Epoch 1/15
2022-07-21 08:38:25.041560: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:38:28.043900: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
178/178 - 158s - loss: 236.4592 - mae: 11.1303 - val_loss: 595.5871 - val_mae: 19.3904
Epoch 2/15
178/178 - 89s - loss: 92.6703 - mae: 7.2277 - val_loss: 722.8016 - val_mae: 21.9107
Epoch 3/15
178/178 - 89s - loss: 68.8762 - mae: 6.2775 - val_loss: 298.3092 - val_mae: 12.9265
Epoch 4/15
178/178 - 89s - loss: 57.1186 - mae: 5.7820 - val_loss: 132.9691 - val_mae: 8.8124
Epoch 5/15
178/178 - 89s - loss: 48.0338 - mae: 5.3353 - val_loss: 87.0461 - val_mae: 6.8346
Epoch 6/15
178/178 - 89s - loss: 41.4338 - mae: 4.9560 - val_loss: 73.4758 - val_mae: 6.2678
Epoch 7/15
178/178 - 89s - loss: 35.8353 - mae: 4.5782 - val_loss: 68.6192 - val_mae: 6.1127
Epoch 8/15
178/178 - 89s - loss: 32.0619 - mae: 4.3735 - val_loss: 89.2643 - val_mae: 7.2203
Epoch 9/15
178/178 - 89s - loss: 28.9610 - mae: 4.1563 - val_loss: 65.9167 - val_mae: 6.0166
Epoch 10/15
178/178 - 89s - loss: 26.2554 - mae: 3.9268 - val_loss: 66.4694 - val_mae: 6.1470
Epoch 11/15
178/178 - 89s - loss: 22.8827 - mae: 3.6888 - val_loss: 77.8586 - val_mae: 6.7697
Epoch 12/15
178/178 - 89s - loss: 20.6356 - mae: 3.4994 - val_loss: 70.5316 - val_mae: 6.3521
Epoch 13/15
178/178 - 89s - loss: 19.2414 - mae: 3.3738 - val_loss: 66.9636 - val_mae: 6.0657
Epoch 14/15
178/178 - 89s - loss: 16.9266 - mae: 3.1803 - val_loss: 65.0013 - val_mae: 5.7957
Epoch 15/15
178/178 - 89s - loss: 15.5948 - mae: 3.0519 - val_loss: 65.8877 - val_mae: 5.8612
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 10s - loss: 65.8877 - mae: 5.8612
Test MAE: 5.8612
2022-07-21 08:37:33.998052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2022-07-21 08:37:34.318023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Using TensorFlow backend.
Found 5694 validated image filenames.
Found 1897 validated image filenames.
2022-07-21 08:37:45.066868: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-07-21 08:37:45.779578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-21 08:37:45.779678: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:45.779710: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:37:46.176033: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-21 08:37:46.280479: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-21 08:37:47.040109: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-21 08:37:47.091263: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2022-07-21 08:37:47.091410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-07-21 08:37:47.096102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2022-07-21 08:37:47.096507: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-21 08:37:47.305573: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2022-07-21 08:37:47.308022: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x489e9f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-21 08:37:47.308055: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-21 08:37:47.665354: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4924a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-07-21 08:37:47.665394: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2022-07-21 08:37:47.667882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-21 08:37:47.667944: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:47.667954: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:37:47.667993: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-21 08:37:47.668004: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-21 08:37:47.668014: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-21 08:37:47.668024: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2022-07-21 08:37:47.668032: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-07-21 08:37:47.672478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2022-07-21 08:37:47.685182: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:52.829148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-21 08:37:52.829193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2022-07-21 08:37:52.829201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2022-07-21 08:37:52.843079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 30509 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 178 steps, validate for 60 steps
Epoch 1/15
2022-07-21 08:38:25.041560: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:38:28.043900: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
178/178 - 158s - loss: 236.4592 - mae: 11.1303 - val_loss: 595.5871 - val_mae: 19.3904
Epoch 2/15
178/178 - 89s - loss: 92.6703 - mae: 7.2277 - val_loss: 722.8016 - val_mae: 21.9107
Epoch 3/15
178/178 - 89s - loss: 68.8762 - mae: 6.2775 - val_loss: 298.3092 - val_mae: 12.9265
Epoch 4/15
178/178 - 89s - loss: 57.1186 - mae: 5.7820 - val_loss: 132.9691 - val_mae: 8.8124
Epoch 5/15
178/178 - 89s - loss: 48.0338 - mae: 5.3353 - val_loss: 87.0461 - val_mae: 6.8346
Epoch 6/15
178/178 - 89s - loss: 41.4338 - mae: 4.9560 - val_loss: 73.4758 - val_mae: 6.2678
Epoch 7/15
178/178 - 89s - loss: 35.8353 - mae: 4.5782 - val_loss: 68.6192 - val_mae: 6.1127
Epoch 8/15
178/178 - 89s - loss: 32.0619 - mae: 4.3735 - val_loss: 89.2643 - val_mae: 7.2203
Epoch 9/15
178/178 - 89s - loss: 28.9610 - mae: 4.1563 - val_loss: 65.9167 - val_mae: 6.0166
Epoch 10/15
178/178 - 89s - loss: 26.2554 - mae: 3.9268 - val_loss: 66.4694 - val_mae: 6.1470
Epoch 11/15
178/178 - 89s - loss: 22.8827 - mae: 3.6888 - val_loss: 77.8586 - val_mae: 6.7697
Epoch 12/15
178/178 - 89s - loss: 20.6356 - mae: 3.4994 - val_loss: 70.5316 - val_mae: 6.3521
Epoch 13/15
178/178 - 89s - loss: 19.2414 - mae: 3.3738 - val_loss: 66.9636 - val_mae: 6.0657
Epoch 14/15
178/178 - 89s - loss: 16.9266 - mae: 3.1803 - val_loss: 65.0013 - val_mae: 5.7957
Epoch 15/15
178/178 - 89s - loss: 15.5948 - mae: 3.0519 - val_loss: 65.8877 - val_mae: 5.8612
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 10s - loss: 65.8877 - mae: 5.8612
Test MAE: 5.8612
2022-07-21 08:37:33.998052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2022-07-21 08:37:34.318023: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Using TensorFlow backend.
Found 5694 validated image filenames.
Found 1897 validated image filenames.
2022-07-21 08:37:45.066868: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-07-21 08:37:45.779578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-21 08:37:45.779678: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:45.779710: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:37:46.176033: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-21 08:37:46.280479: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-21 08:37:47.040109: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-21 08:37:47.091263: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2022-07-21 08:37:47.091410: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-07-21 08:37:47.096102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2022-07-21 08:37:47.096507: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-21 08:37:47.305573: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2022-07-21 08:37:47.308022: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x489e9f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-07-21 08:37:47.308055: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-07-21 08:37:47.665354: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4924a80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-07-21 08:37:47.665394: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2022-07-21 08:37:47.667882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:8b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-21 08:37:47.667944: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:47.667954: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:37:47.667993: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-07-21 08:37:47.668004: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-07-21 08:37:47.668014: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolver.so.10
2022-07-21 08:37:47.668024: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusparse.so.10
2022-07-21 08:37:47.668032: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-07-21 08:37:47.672478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1697] Adding visible gpu devices: 0
2022-07-21 08:37:47.685182: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-07-21 08:37:52.829148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1096] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-07-21 08:37:52.829193: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1102]      0 
2022-07-21 08:37:52.829201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1115] 0:   N 
2022-07-21 08:37:52.843079: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 30509 MB memory) -> physical GPU (device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0)
<class 'tensorflow.python.keras.engine.sequential.Sequential'>
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
Train for 178 steps, validate for 60 steps
Epoch 1/15
2022-07-21 08:38:25.041560: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-21 08:38:28.043900: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
178/178 - 158s - loss: 236.4592 - mae: 11.1303 - val_loss: 595.5871 - val_mae: 19.3904
Epoch 2/15
178/178 - 89s - loss: 92.6703 - mae: 7.2277 - val_loss: 722.8016 - val_mae: 21.9107
Epoch 3/15
178/178 - 89s - loss: 68.8762 - mae: 6.2775 - val_loss: 298.3092 - val_mae: 12.9265
Epoch 4/15
178/178 - 89s - loss: 57.1186 - mae: 5.7820 - val_loss: 132.9691 - val_mae: 8.8124
Epoch 5/15
178/178 - 89s - loss: 48.0338 - mae: 5.3353 - val_loss: 87.0461 - val_mae: 6.8346
Epoch 6/15
178/178 - 89s - loss: 41.4338 - mae: 4.9560 - val_loss: 73.4758 - val_mae: 6.2678
Epoch 7/15
178/178 - 89s - loss: 35.8353 - mae: 4.5782 - val_loss: 68.6192 - val_mae: 6.1127
Epoch 8/15
178/178 - 89s - loss: 32.0619 - mae: 4.3735 - val_loss: 89.2643 - val_mae: 7.2203
Epoch 9/15
178/178 - 89s - loss: 28.9610 - mae: 4.1563 - val_loss: 65.9167 - val_mae: 6.0166
Epoch 10/15
178/178 - 89s - loss: 26.2554 - mae: 3.9268 - val_loss: 66.4694 - val_mae: 6.1470
Epoch 11/15
178/178 - 89s - loss: 22.8827 - mae: 3.6888 - val_loss: 77.8586 - val_mae: 6.7697
Epoch 12/15
178/178 - 89s - loss: 20.6356 - mae: 3.4994 - val_loss: 70.5316 - val_mae: 6.3521
Epoch 13/15
178/178 - 89s - loss: 19.2414 - mae: 3.3738 - val_loss: 66.9636 - val_mae: 6.0657
Epoch 14/15
178/178 - 89s - loss: 16.9266 - mae: 3.1803 - val_loss: 65.0013 - val_mae: 5.7957
Epoch 15/15
178/178 - 89s - loss: 15.5948 - mae: 3.0519 - val_loss: 65.8877 - val_mae: 5.8612
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 10s - loss: 65.8877 - mae: 5.8612
Test MAE: 5.8612